THINGS TO DO TO UPDATE WEBSITE: <br>
jupyter nbconvert --to html "Final Tutorial.ipynb" <br>
delete old index.html, rename "Final Tutorial.html" to index.html <br>
rename title in file to "Analysis of NFL Play by Play Data"

# BEGIN
# ---------------------------------------------------------------------------------------------------------------

# Analysis of NFL Play by Play Data
Pierson Polcaro

# Introduction

https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016 <br>
if necessary^

# \<import\>

In [1]:
import numpy as np
import pandas as pd

# Data Collection

In [2]:
# Read data in chunks so it fits in memory
chunks = pd.read_csv("NFL Play by Play 2009-2017.csv", chunksize=1000)
data_master = pd.concat(chunks)
data_master.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


# Data Processing

# Data Visualization

# Analysis

# Insight